In [8]:
import numpy as np
import pandas as pd
import sklearn.datasets

In [3]:
names_genre = np.load('./namesngenre.npy')
ratings_train = np.load('./ratings_train.npy')

df_titles = pd.DataFrame(names_genre)
df_titles= df_titles.rename(columns={0:"title",1:"genre"})
df_titles['average_rating'] = np.nanmean(ratings_train, axis=0)

df_ratings = pd.DataFrame(ratings_train)
ratings_train.shape

(610, 4980)

In [4]:
df_titles

,title,genre,average_rating
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.854167
1,Jumanji (1995),Adventure|Children|Fantasy,3.391892
2,Grumpier Old Men (1995),Comedy|Romance,3.029412
3,Waiting to Exhale (1995),Comedy|Drama|Romance,2.000000
4,Father of the Bride Part II (1995),Comedy,3.000000
...,...,...,...
4975,Game Night (2018),Action|Comedy|Crime|Horror,4.500000
4976,Making a Murderer (2015),Crime|Documentary,3.500000
4977,Elsa & Fred (2005),Comedy|Drama|Romance,3.750000
4978,Tomb Raider (2018),Action|Adventure|Fantasy,3.500000


In [6]:
df_ratings.to_csv('./documents/ratings_train.csv')

In [ ]:
sklearn.datasets.make_low_rank_matrix()